In [1]:
import pandas as pd

In [77]:
cpuc = pd.read_csv('data/cpuc_with_budget_and_resource.csv')

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,12,17,19,22,23,24,27,47,48,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Want to make a column that shows percentage of budget of resource claims/obf claims in the final aggregated dataframe

In [179]:
def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
    # creating a total claims column
    
    df['Total Claims'] = [1]*len(df)
    
    #aggregate!
    relevant_columns = column_list + ['Total Claims']
    agg = df.groupby(by).sum()[relevant_columns]
    agg['Total Programs'] = tp
    
    # Resource/OBF budget as percentage of total budget
    obf_budget = yay.groupby(['SiteCity', 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
    resource_budget = yay.groupby(['SiteCity', 'Resource_Flag']).Budget.sum().unstack().fillna(0).loc[:, 1]
    agg['% OBF Budget'] = (obf_budget/agg['Budget'])*100
    agg['% Resource Budget'] = (resource_budget/agg['Budget'])*100
    
    return(agg.reset_index())

In [123]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    dupes = all_dupes['index'].to_list()
   

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    
    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    return(df.drop(index=badi).reset_index())

In [124]:
copy = cpuc.copy()

copy.drop(columns='Unnamed: 0', inplace=True)

copy = copy.drop_duplicates().reset_index()

yay = no_dupes(copy)

In [148]:
ag = aggregate_me(yay, 'SiteCity', False, ['Budget', 'Resource_Flag', 'OBF_Flag'])
ag

,Budget,Resource_Flag,OBF_Flag,Total Claims,Total Programs
SiteCity,,,,,
ACAMPO,4.031116e+04,15,0,15,2
ACTON,1.157423e+05,62,0,62,4
ADELANTO,8.736705e+05,568,0,568,7
AGOURA,7.469575e+03,8,0,8,1
AGOURA HILLS,2.134583e+05,115,0,115,7
...,...,...,...,...,...
YOSEMITE NATIONAL PARK,2.547790e+04,11,0,11,2
YOUNTVILLE,2.665221e+04,80,0,80,3
YUBA CITY,1.378842e+06,1046,52,1046,8


In [149]:
ag[ag['Resource_Flag']!=ag['Total Claims']]

,Budget,Resource_Flag,OBF_Flag,Total Claims,Total Programs
SiteCity,,,,,
LOS ANGELES,2.171608e+07,6767,1,7163,62
ROSEMEAD,1.510533e+07,579,1,594,56
SAN DIEGO,8.076950e+07,42481,371,42813,38
SAN FRANCISCO,7.128608e+07,9224,249,9236,42


### *The following work assumes claims can either be resource/obf or not, and the difference between the total claims and the resource/obf claims is equal to the non-resource/non-obf claims*

In [165]:
rb = yay.groupby(['SiteCity', 'Resource_Flag']).Budget.sum()
rb

SiteCity                Resource_Flag
ACAMPO                  1                4.031116e+04
ACTON                   1                1.157423e+05
ADELANTO                1                8.736705e+05
AGOURA                  1                7.469575e+03
AGOURA HILLS            1                2.134583e+05
                                             ...     
YOSEMITE NATIONAL PARK  1                2.547790e+04
YOUNTVILLE              1                2.665221e+04
YUBA CITY               1                1.378842e+06
YUCAIPA                 1                4.099928e+05
YUCCA VALLEY            1                1.664178e+05
Name: Budget, Length: 1030, dtype: float64

### *from the above it looks like this may not be true, because I would think that the two budgets, resource or not, would add up to the total budget, but this is not the case*
it also might be a lack of my understanding of groupby

Luckily, for most of the cities, resource claims make up 100% of the budget, it's just for the four cities above that this is an issue. 
But those 3/4 cities above are the largest in the state....

### *jk was just because I forgot that I had only included local gov programs before* so its **ALL GOOD**
now I need to figure out how to get only the 1  parts of the muliindex

In [168]:
resource_budget = rb.unstack().loc[:,1]
resource_budget

SiteCity
ACAMPO                    4.031116e+04
ACTON                     1.157423e+05
ADELANTO                  8.736705e+05
AGOURA                    7.469575e+03
AGOURA HILLS              2.134583e+05
                              ...     
YOSEMITE NATIONAL PARK    2.547790e+04
YOUNTVILLE                2.665221e+04
YUBA CITY                 1.378842e+06
YUCAIPA                   4.099928e+05
YUCCA VALLEY              1.664178e+05
Name: 1, Length: 1026, dtype: float64

In [173]:
obf_budget = yay.groupby(['SiteCity', 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
obf_budget

SiteCity
ACAMPO                         0.000000
ACTON                          0.000000
ADELANTO                       0.000000
AGOURA                         0.000000
AGOURA HILLS                   0.000000
                              ...      
YOSEMITE NATIONAL PARK         0.000000
YOUNTVILLE                     0.000000
YUBA CITY                 609352.291504
YUCAIPA                        0.000000
YUCCA VALLEY                   0.000000
Name: True, Length: 1026, dtype: float64

In [176]:
ag['% OBF Budget'] = (obf_budget/ag['Budget'])*100
ag['% Resource Budget'] = (resource_budget/ag['Budget'])*100
ag

,Budget,Resource_Flag,OBF_Flag,Total Claims,Total Programs,% OBF,% OBF Budget,% Resource Budget
SiteCity,,,,,,,,
ACAMPO,4.031116e+04,15,0,15,2,0.000000,0.000000,100.0
ACTON,1.157423e+05,62,0,62,4,0.000000,0.000000,100.0
ADELANTO,8.736705e+05,568,0,568,7,0.000000,0.000000,100.0
AGOURA,7.469575e+03,8,0,8,1,0.000000,0.000000,100.0
AGOURA HILLS,2.134583e+05,115,0,115,7,0.000000,0.000000,100.0
...,...,...,...,...,...,...,...,...
YOSEMITE NATIONAL PARK,2.547790e+04,11,0,11,2,0.000000,0.000000,100.0
YOUNTVILLE,2.665221e+04,80,0,80,3,0.000000,0.000000,100.0
YUBA CITY,1.378842e+06,1046,52,1046,8,0.441931,44.193056,100.0


In [180]:
aggregate_me(yay, 'SiteCity', True, ['Budget'])

<ipython-input-179-fd46151a01d0>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total Claims'] = [1]*len(df)


,SiteCity,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget
0,ACAMPO,17684.568509,6,1,0.000000,227.945370
1,ADELANTO,27320.063231,4,1,0.000000,3197.907943
2,ALAMO,48785.542683,15,1,88.044896,253.582889
3,ALBANY,60926.193528,35,2,5.162880,234.743046
4,ALHAMBRA,48465.447601,7,1,0.000000,1768.790160
...,...,...,...,...,...,...
532,WOODLAND,278715.826268,358,2,72.843072,543.971893
533,YORBA LINDA,230594.633505,12,1,100.000000,360.667037
534,YOSEMITE NATIONAL PARK,18234.382541,4,1,0.000000,139.724500
535,YOUNTVILLE,4901.835174,57,2,0.000000,543.719042


whooooo